In [89]:
import numpy as np
dataset = np.load("datasets/dataset_moves.npy", allow_pickle=True)
results = np.load("datasets/dataset_results.npy", allow_pickle=True)
names = np.load("datasets/dataset_names.npy", allow_pickle=True)

In [90]:
# Check if all the matches are equal
indices_equal = []
for i in range(len(dataset)):
    for j in range(i + 1, len(dataset)):
        if  np.array_equal(dataset[i], dataset[j]) and not ((j,i) in indices_equal):
            indices_equal.append((i, j))
            
print(f"Number of equal matches: {len(indices_equal)}")

print(f"the eqal matches are:")
for ind, (i,j) in enumerate(indices_equal):
    print(f"\t{ind+1}) {names[i]}\tand\t{names[j]}")
    

Number of equal matches: 0
the eqal matches are:


## Quanti match utili

In [91]:
X = 15
count = sum(len(match) > X for match in dataset)
print(f"Nel dataset ci sono {count} match con più di {X} mosse")

Nel dataset ci sono 9 match con più di 15 mosse


## La n-esima mossa più frequente

In [92]:
from collections import Counter

idx = 0         # which move (column)
num_list = 5    # how many results

idx_moves = []
for match in dataset :
    try:
        idx_moves.append(tuple(match[idx]))
    except: continue
    
move_counter = Counter(idx_moves)
most_common_move, most_common_count = move_counter.most_common(1)[0]

most_common_moves = move_counter.most_common(num_list)

# Stampa le N mosse più frequenti
for move, count in most_common_moves:
    print(f"Mossa: ({move[0]}, {move[1]}), Occorrenze: {count}")

Mossa: (g5, g9), Occorrenze: 12
Mossa: (e6, d6), Occorrenze: 3
Mossa: (e6, f6), Occorrenze: 1
Mossa: (f5, f4), Occorrenze: 1
Mossa: (c5, c1), Occorrenze: 1


## Quante vittorie, pareggi o sconfitte ?

In [93]:
white_wins_supposed = sum(len(match)%2!=0 for match in dataset)
black_wins_supposed = sum(len(match)%2==0 for match in dataset)

white_wins = sum(res=="W" for res in results)   # White wins
black_wins = sum(res=="B" for res in results)   # Black wins
draws =      sum(res=="D" for res in results)   # Draw
other =      sum(res=="I" for res in results)   # Incomplete result

print(f"Total matches: {len(dataset)}")

# print(white_wins_supposed,black_wins_supposed)
# print(white_wins_supposed + black_wins_supposed)

print(f"\twhite_wins={white_wins}\n\tblack_wins={black_wins}\n\tdraws={draws}\n\tother={other}")
# print(white_wins + black_wins + draws + other)

Total matches: 19
	white_wins=13
	black_wins=1
	draws=5
	other=0


In [94]:
incomplete_indices = [i for i, res in enumerate(results) if res == "I"]

# Stampa gli indici delle partite incomplete
print("Incomplete matches:")
for i in incomplete_indices:
    print(f"\t{names[i]}")

Incomplete matches:


## Delete incomplete results

In [95]:
incomplete_idxs = results=="I"
dataset_cleaned = np.delete(dataset, incomplete_idxs, axis=0)
results_cleaned = np.delete(results, incomplete_idxs, axis=0)

In [96]:
white_wins_supposed = sum(len(match)%2!=0 for match in dataset_cleaned)
black_wins_supposed = sum(len(match)%2==0 for match in dataset_cleaned)

white_wins = sum(res=="W" for res in results_cleaned)   # White wins
black_wins = sum(res=="B" for res in results_cleaned)   # Black wins
draws =      sum(res=="D" for res in results_cleaned)   # Draw
other =      sum(res=="I" for res in results_cleaned)   # Incomplete result

print(f"Total matches: {len(dataset)}")

# print(white_wins_supposed,black_wins_supposed)
# print(white_wins_supposed + black_wins_supposed)

print(f"\twhite_wins={white_wins}\n\tblack_wins={black_wins}\n\tdraws={draws}\n\tother={other}")
# print(white_wins + black_wins + draws + other)

Total matches: 19
	white_wins=13
	black_wins=1
	draws=5
	other=0


In [97]:
# np.save("dataset_moves_cleaned.npy", dataset_cleaned)
# np.save("dataset_results_cleaned.npy", results_cleaned)

## Qual è la n-esima mossa più vincente ?

In [98]:
from collections import Counter

idx = 2         # which move (column)
num_list = 5    # how many results (None for all)

idx_moves = []
for i, match in enumerate(dataset_cleaned) :
    try:
        res = results_cleaned[i]
        if (idx % 2 == 0 and res=="W") or (idx % 2 != 0 and res=="B") :
            idx_moves.append(tuple(match[idx]))
    except: continue
    
move_counter = Counter(idx_moves)
most_common_move, most_common_count = move_counter.most_common(1)[0]

most_common_moves = move_counter.most_common(num_list)

# Stampa le N mosse più frequenti
# c = 0   # probability check
for move, count in most_common_moves:
    
    if idx % 2 == 0:
        print(f"Mossa: ({move[0].item()}, {move[1].item()}), Win race: {count / white_wins * 100:.4f}%")
        # c += count / white_wins * 100   # probability check
    else:
        print(f"Mossa: ({move[0].item()}, {move[1].item()}), Win race: {count / black_wins * 100:.4f}%")
        # c += count / black_wins * 100   # probability check
# print(c)    # probability check

Mossa: (g9, i9), Win race: 23.0769%
Mossa: (g9, g1), Win race: 15.3846%
Mossa: (e5, f5), Win race: 15.3846%
Mossa: (f5, g5), Win race: 7.6923%
Mossa: (f5, f8), Win race: 7.6923%


In [ ]:
import pandas as pd

names_parsed = [(name.split("_")[1], name.split("_")[3]) for name in names]

players = set(player for pair in names_parsed for player in pair)

correlation_matrix = pd.DataFrame("", index=list(players), columns=list(players), dtype=str)

for idx, (p1, p2) in enumerate(names_parsed):
    if results[idx] == "W":  
        correlation_matrix.at[p1, p2] += "W"
    elif results[idx] == "B":  
        correlation_matrix.at[p1, p2] += "B"
    else:  
        correlation_matrix.at[p1, p2] += "D"
        # correlation_matrix.at[p2, p1] += "D"

print(correlation_matrix)

       flag3 grey flag2 flag1 random
flag3           W     W     W      W
grey       W          D            W
flag2      B    D           D      W
flag1      W    W     W            W
random     D    D     W     W       
